# 1. Instalações e importações

In [1]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split # separa o dataset de teste
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
# from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
# from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # matriz de confusão
import matplotlib.pyplot as plt

# 2. Importação dos dados
O dataframe original conta com 474230 dados - em relação às compras na América do Norte -, mas foi reduzido para uma amostra de 100.000 dados escolhidos de forma aleatória, para fins de estudo.

In [5]:
data = "./data/amostraNORAM.csv"


In [6]:
test_data = "./data/testesNORAM.csv"

# 3. Análise Inicial


O modelo desevolvido tem como alvo a classificação das compras em relação ao Level 2, visto que essa foi a principal dor apresentada pelo parceiro.

In [7]:
dataframe = pd.read_csv(data)
dataframe.head(5)

,Supplier Name,Normalized Supplier Name,Parent Supplier Name,Region,Country Name,Strategic Region,Requestor Name,Preparer Name,Level 1,Level 2,...,GL Desc (Level 6),Invoice ID,Invoice Number,Invoice Source,GL Description,Product,Project,"Month, Day, Year of Payment Date",PO Number,Amout USD
0,BEELINECOM INC,BEELINECOM INC,"Beeline.com, LLC",NORAM,United States of America,NORAM,NaN,NaN,Technology/Telecom,Software & Support,...,Operating Expenses w/o Allocations,300002627497274,BEELINE42216639751,BEELINE,Temporary services,NaN,53033 - Infra Silicon,2023-03-09,NaN,30
1,RUSSELL TOBIN AND ASSOCIATES,RUSSELL TOBIN AND ASSOCIATES LLC,"Pride Capital Partners, LLC",NORAM,United States of America,NORAM,NaN,NaN,Human Resources,Temporary Personnel Services,...,Operating Expenses w/o Allocations,300002671575174,42256848991,BEELINE,Temporary services,NaN,10210 - OS - Other,2023-03-25,NaN,4382
2,BEELINECOM INC,BEELINECOM INC,"Beeline.com, LLC",NORAM,United States of America,NORAM,NaN,NaN,Technology/Telecom,Software & Support,...,Operating Expenses w/o Allocations,300002543611199,BEELINE35516355961,BEELINE,Temporary services,NaN,43234 - XR People,2023-01-26,NaN,18
3,AMAZON.COM LLC,AMAZON,Amazon.Com LLC,NORAM,United States of America,NORAM,Kimberly Childress,Kimberly Childress,Technology/Telecom,Computer Hardware,...,Operating Expenses w/o Allocations,300002666642479,1DLP3RRJ4JRX,LETTERBOX,Miscellaneous expenses,Default Product,00000 - Default Project,2023-04-06,7.000065e+10,53
4,NORTHLAND CONTROL SYSTEMS INC,NORTHLAND CONTROL SYSTEMS INC,Northland Control Systems Inc,NORAM,United States of America,NORAM,Julie Bastoky,Shannon Poe,Real Estate & Facilities,Security,...,Cost of Goods Sold,300002682193303,37237,SUPP_CONNECT,Data Center – Colo Common Area Maintenance,Default Product,31094 - GSST Global Security System Technology,2023-04-15,7.000062e+10,17000


In [8]:
dataframe.shape

(100000, 30)

### Definição da função

In [13]:
def drop_columns(dados):
    result = dados.drop(['Invoice ID', 'Invoice Number', 'Supplier Name', 'Requestor Name', 'Preparer Name', 'PO Number', 'Region', 'Level 3', 'Month, Day, Year of Payment Date', 'Parent Supplier Name', 'Region', 'Strategic Region', 'Cost Center (Level 6)', 'GL Desc (Level 5)', 'GL Desc (Level 6)', 'Amout USD', 'Country Name',  'Cost Center','Cost Center (Level 4)', 'Cost Center (Base Level)', 'Cost Center (Level 5)'], axis=1)
    return result

### Aplicação ao dataframe

In [14]:
df = drop_columns(dataframe)
df.head(5)

,Normalized Supplier Name,Level 1,Level 2,Business Unit,Legal Entity,GL Desc (Level 4),Invoice Source,GL Description,Product,Project
0,BEELINECOM INC,Technology/Telecom,Software & Support,Infrastructure,"Meta Platforms, Inc.",Temporary services,BEELINE,Temporary services,NaN,53033 - Infra Silicon
1,RUSSELL TOBIN AND ASSOCIATES LLC,Human Resources,Temporary Personnel Services,IT & Corpnet,"Meta Platforms, Inc.",Temporary services,BEELINE,Temporary services,NaN,10210 - OS - Other
2,BEELINECOM INC,Technology/Telecom,Software & Support,RL,"Meta Platforms, Inc.",Temporary services,BEELINE,Temporary services,NaN,43234 - XR People
3,AMAZON,Technology/Telecom,Computer Hardware,RL,"Meta Platforms Technologies, LLC",Miscellaneous expenses,LETTERBOX,Miscellaneous expenses,Default Product,00000 - Default Project
4,NORTHLAND CONTROL SYSTEMS INC,Real Estate & Facilities,Security,Facilities,"Meta Platforms, Inc.",Data Center - Colo Common Area Maintenance,SUPP_CONNECT,Data Center – Colo Common Area Maintenance,Default Product,31094 - GSST Global Security System Technology


In [15]:
df.shape

(100000, 10)

# 4. Pré-modelagem

## 4.1 One hot encoder

### Definição da função

In [16]:
def onehot(dados):
    # Inicializar o OneHotEncoder
    encoder = OneHotEncoder(sparse=False)

    # Aplicar one-hot encoding a todas as colunas categóricas
    result = pd.get_dummies(dados)

    return result

### Aplicação ao dataframe

In [17]:
data_sem_level2 = df.drop('Level 2', axis=1)

df_encoded = onehot(data_sem_level2)
df_encoded

,Normalized Supplier Name_100 PERCENT GROUP LIMITED,Normalized Supplier Name_100 THIEVES,Normalized Supplier Name_1000HEADS INC,Normalized Supplier Name_10UP,Normalized Supplier Name_1547 CSR PITTOCK BLOCK LLC,Normalized Supplier Name_1623 FARNAM LLC,Normalized Supplier Name_1861 CONSULTING LLC,Normalized Supplier Name_1ST AVE MACHINE USA INC,Normalized Supplier Name_210 N TUCKER TENANT LLC,Normalized Supplier Name_2220 RECORDS LLC,...,Project_98013 - 2137 2022 unbilled,Project_98036 - 2137 DRPI LEG - Research & advocacy on global legislative proposals,Project_98052 - 2137 DRPI DFG - Website Updates for Social Capital Lab; Report Development,Project_98058 - 2137 ADS/MON - External ad tech policy consultant - Chappell & Associates,Project_98064 - 2137 AI POL - AI Research CW - Maeve Ryan,Project_98065 - 2137 AI POL - Outside Counsel (Hintze LLC) - Jevan Hutson/Cameron Cantrell,Project_98069 - 2137 CORE APPS - Stakeholder engagement & support,Project_98072 - 2137 CORE APPS - Encryption,Project_98085 - 2137 RL/PROD - CWs - Product Strategy Support and Leave Coverage,Project_98087 - 2137 RL - RL Advisory Council Members and Operations - US and EU
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
99996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
99997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
99998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
dado_level2 = pd.DataFrame(df, columns=['Level 2'])

df_encoded = pd.concat([df_encoded, dado_level2], axis = 1)
df_encoded

,Normalized Supplier Name_100 PERCENT GROUP LIMITED,Normalized Supplier Name_100 THIEVES,Normalized Supplier Name_1000HEADS INC,Normalized Supplier Name_10UP,Normalized Supplier Name_1547 CSR PITTOCK BLOCK LLC,Normalized Supplier Name_1623 FARNAM LLC,Normalized Supplier Name_1861 CONSULTING LLC,Normalized Supplier Name_1ST AVE MACHINE USA INC,Normalized Supplier Name_210 N TUCKER TENANT LLC,Normalized Supplier Name_2220 RECORDS LLC,...,Project_98036 - 2137 DRPI LEG - Research & advocacy on global legislative proposals,Project_98052 - 2137 DRPI DFG - Website Updates for Social Capital Lab; Report Development,Project_98058 - 2137 ADS/MON - External ad tech policy consultant - Chappell & Associates,Project_98064 - 2137 AI POL - AI Research CW - Maeve Ryan,Project_98065 - 2137 AI POL - Outside Counsel (Hintze LLC) - Jevan Hutson/Cameron Cantrell,Project_98069 - 2137 CORE APPS - Stakeholder engagement & support,Project_98072 - 2137 CORE APPS - Encryption,Project_98085 - 2137 RL/PROD - CWs - Product Strategy Support and Leave Coverage,Project_98087 - 2137 RL - RL Advisory Council Members and Operations - US and EU,Level 2
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Software & Support
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Temporary Personnel Services
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Software & Support
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Computer Hardware
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Security
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Software & Support
99996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Temporary Personnel Services
99997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Temporary Personnel Services
99998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Temporary Personnel Services


## 4.2 Label encoder da Level 2 (alvo)

### Definição da função

In [19]:
def label(dados):
    # Inicializar o LabelEncoder
    label_encoder = LabelEncoder()

    # Aplicar o Label Encoding à coluna 'Level 2'
    result = label_encoder.fit_transform(dados)
    
    return result

### Aplicação ao dataframe

In [20]:
# df_encoded['Level 2'] = label(df_encoded['Level 2'])

df_encoded['Level 2']

0                  Software & Support
1        Temporary Personnel Services
2                  Software & Support
3                   Computer Hardware
4                            Security
                     ...             
99995              Software & Support
99996    Temporary Personnel Services
99997    Temporary Personnel Services
99998    Temporary Personnel Services
99999               Talent Management
Name: Level 2, Length: 100000, dtype: object

## 4.3 - divisão/preparação do dataset
Nessa etapa inicial, escolhemos a features a serem utilizadas no modelo (variavel "x"), e a variável alvo ("y")

In [21]:
dataframe.columns

Index(['Supplier Name', 'Normalized Supplier Name', 'Parent Supplier Name',
       'Region', 'Country Name', 'Strategic Region', 'Requestor Name',
       'Preparer Name', 'Level 1', 'Level 2', 'Level 3', 'Business Unit',
       'Legal Entity', 'Cost Center', 'Cost Center (Base Level)',
       'Cost Center (Level 4)', 'Cost Center (Level 5)',
       'Cost Center (Level 6)', 'GL Desc (Level 4)', 'GL Desc (Level 5)',
       'GL Desc (Level 6)', 'Invoice ID', 'Invoice Number', 'Invoice Source',
       'GL Description', 'Product', 'Project',
       'Month, Day, Year of Payment Date', 'PO Number', 'Amout USD'],
      dtype='object')

In [22]:
x = df_encoded.drop(['Level 2'], axis = 1) #features
y = df_encoded['Level 2'] #alvo

### Divisão dos conjuntos de teste e treino

In [23]:

x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=32)

# 5. Modelo KNN

O KNN é um modelo de classificação que, ao classificar um item de classe desconhecida, leva em conta a classe dos K itens vizinhos mais próximos. Nesse caso, utilizaremos como base para a predição K=13, e faremos o cálculo da distância entre os vizinhos utilizando a distância euclidiana.

In [24]:
#calculo para ajudar a definir o k
import math
math.sqrt(len(y_test))

173.20508075688772

In [25]:
#modelo
neigh = KNeighborsClassifier(n_neighbors=13, metric='euclidean')
neigh.fit(x_train, y_train.squeeze())

KNeighborsClassifier(metric='euclidean', n_neighbors=13)

## Avaliação das predições

In [26]:
y_pred = neigh.predict(x_test)
y_pred

array(['Temporary Personnel Services', 'Temporary Personnel Services',
       'Office Equipment & Services', ..., 'Talent Management',
       'R&D Equipment Services', 'Temporary Personnel Services'],
      dtype=object)

### Acurácia de treino e teste

In [27]:
print('Acuracia (treino): ', neigh.score( x_train, y_train ))

Acuracia (treino):  0.9451142857142857


In [28]:
print('Acuracia (teste): ', neigh.score( x_test, y_test ))

Acuracia (teste):  0.9336


# 6. Pipeline de testes

### pipeline de pré-processamento

In [29]:
def pipeline(dado):
    dado = pd.read_csv(dado)
    dado = pd.concat([dataframe, dado], ignore_index=True) #essa etapa é necessaria para que as colunas criadas pelo one hot encoding sejam as mesmas
    dado = drop_columns(dado)
    dado_x = dado.drop('Level 2', axis=1)
    dado_x = onehot(dado_x)
    dado_y = pd.DataFrame(dado, columns=['Level 2'])
    dado = pd.concat([dado_x, dado_y], axis = 1)
    dado = dado.tail(10)
    return dado

### pipeline de classificação

In [30]:
def pipe(dado):
    x_test_2 = dado.drop(['Level 2'], axis = 1) #features
    y_test_2 = dado['Level 2'] #alvo 
    y_pred_2 = neigh.predict(x_test_2)
    print(y_pred_2)
    # printar acuracia
    print('Acuracia (entre os 10 testes isolados): ', neigh.score( x_test_2, y_test_2 ))
    
    return y_pred_2


## Testes isolados do modelo de classificação

primeiro, vamos visualizar a classificação original e correta dos 10 dados de amostra, que foram retirados e não estão presentes no conjunto de treino nem testes anterior.

In [31]:
testes = pipeline(test_data)

In [32]:
testes['Level 2']

100000               Expedited Freight
100001               Talent Management
100002              Software & Support
100003               Talent Management
100004           Outsourced Operations
100005    Temporary Personnel Services
100006               Talent Management
100007                 Food & Beverage
100008               Computer Hardware
100009               Talent Management
Name: Level 2, dtype: object

In [33]:
resultado = pipe(testes)

['Expedited Freight' 'Talent Management' 'Software & Support'
 'Talent Management' 'Outsourced Operations'
 'Temporary Personnel Services' 'Talent Management' 'Food & Beverage'
 'Computer Hardware' 'Talent Management']
Acuracia (entre os 10 testes isolados):  1.0


### matriz de confusão

In [34]:
cm = confusion_matrix(y_test_2, y_pred_2)
print(cm)

NameError: name 'y_test_2' is not defined

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot()

### Gráficos e análises

In [ ]:
# Número total de dados
total_data = 30000

# Porcentagem de dados classificados corretamente
accuracy_percentage = 69

# Calcular o número de dados classificados corretamente
correctly_classified = int((accuracy_percentage / 100) * total_data)

# Calcular o número de dados classificados incorretamente
incorrectly_classified = total_data - correctly_classified

# Rótulos e valores para o gráfico de barras
labels = ['Classificado Corretamente', 'Classificado Incorretamente']
values = [correctly_classified, incorrectly_classified]

# Criar o gráfico de barras
plt.bar(labels, values, color=['green', 'red'])
plt.ylabel('Número de Dados')
plt.title('Classificação de Dados')
plt.show()

In [ ]:
# Número total de dados
total_data = 10

# Porcentagem de dados classificados corretamente
accuracy_percentage = 80

# Calcular o número de dados classificados corretamente
correctly_classified = int((accuracy_percentage / 100) * total_data)

# Calcular o número de dados classificados incorretamente
incorrectly_classified = total_data - correctly_classified

# Gerar dados de exemplo para o gráfico de dispersão
np.random.seed(42)
correct_data = np.random.rand(correctly_classified, 2)
incorrect_data = np.random.rand(incorrectly_classified, 2)

# Criar o gráfico de dispersão
plt.scatter(correct_data[:, 0], correct_data[:, 1], color='green', label='Classificado Corretamente')
plt.scatter(incorrect_data[:, 0], incorrect_data[:, 1], color='red', label='Classificado Incorretamente')

# Adicionar rótulos e legenda
plt.xlabel('Eixo X')
plt.ylabel('Eixo Y')
plt.title('Gráfico de Dispersão - Classificação de Dados')
plt.legend()

# Exibir o gráfico
plt.show()

In [ ]:

# Dados fictícios de exemplo
porcentagem_acerto = 80
porcentagem_erro = 20

# Crie um DataFrame para os dados
data = pd.DataFrame({'Acerto (%)': [porcentagem_acerto], 'Erro (%)': [porcentagem_erro]})

# Plote o gráfico de dispersão
data.plot(x='Acerto (%)', y='Erro (%)', kind='scatter', title='Gráfico de Dispersão', grid=True)
plt.xlabel('Porcentagem de Acerto')
plt.ylabel('Porcentagem de Erro')
plt.show()